In [ ]:
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
from pathlib import Path

IMAGES_PATH = Path() / "images" / "DailyResourceConsumption"
IMAGES_PATH.mkdir(parents=True, exist_ok=True)

def save_fig(fig_id, tight_layout=True, fig_extension="png", resolution=300):
    path = IMAGES_PATH / f"{fig_id}.{fig_extension}"
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, format=fig_extension, dpi=resolution)

In [ ]:
from pathlib import Path

orgin_path = "D:\\Iman\\AGHC\\CherryTomato\Data\\"
filenames = ['AICU', 'Automatoes',
             'Digilog', 'IUACAAS', 
             'Reference', 'TheAutomators'
             ]
daily_Avg=dict()
Daily_Resource_Consumption=dict()
greenhouse_climate=dict()
for filename in filenames:
    path=Path(orgin_path+filename+'\\Resources.csv')
    Daily_Resource_Consumption[filename] = pd.read_csv(path,parse_dates=["%Time "],low_memory=False).set_index('%Time ').astype(float)
    #print(Daily_Resource_Consumption[filename].index.shape)
    Daily_Resource_Consumption[filename]['%Time_t']=Daily_Resource_Consumption[filename].index.astype(str)+"_"+filename
    Daily_Resource_Consumption[filename]=Daily_Resource_Consumption[filename].set_index('%Time_t')
    path=Path(orgin_path+filename+'\\GreenhouseClimate.csv')
    greenhouse_climate[filename]=pd.read_csv(path,parse_dates=["%time"],low_memory=False).set_index('%time').astype(float)
    
    #greenhouse_climate[filename].drop(columns=[  'AssimLight', 'BlackScr', 'CO2air', 'EC_drain_PC', 'EnScr', 'HumDef',
    #    'Rhair','Tot_PAR', 'Tot_PAR_Lamps','VentLee', 'Ventwind', 'co2_dos'],inplace=True)
    sp_vip_paired_attributtes=[ [('assim_sp','assim_vip'),('assim_vip','assim_sp')],
                            [('co2_sp','co2_vip'),('co2_vip','co2_sp')],
                            [('dx_sp','dx_vip'),('dx_vip','dx_sp')],
                            [('int_blue_sp','int_blue_vip'),('int_blue_vip','int_blue_sp')],
                            [('int_farred_sp','int_farred_vip'),('int_farred_vip','int_farred_sp')],
                            [('int_red_sp','int_red_vip'),('int_red_vip','int_red_sp')], 
                            [('int_white_sp','int_white_vip'),('int_white_vip','int_white_sp')],
                            [('scr_blck_sp','scr_blck_vip'),('scr_blck_vip','scr_blck_sp')],
                            [('scr_enrg_sp','scr_enrg_vip'),('scr_enrg_vip','scr_enrg_sp')],
                            [('t_grow_min_sp','t_grow_min_vip'),('t_grow_min_vip','t_grow_min_sp')],
                            [('t_heat_sp','t_heat_vip'),('t_heat_vip','t_heat_sp')],
                            [('t_rail_min_sp','t_rail_min_vip'),('t_rail_min_vip','t_rail_min_sp')],
                            [('t_vent_sp','t_ventlee_vip'),('t_ventlee_vip','t_vent_sp')],
                            [('water_sup_intervals_sp_min','water_sup_intervals_vip_min'),('water_sup_intervals_vip_min','water_sup_intervals_sp_min')],
                            [('window_pos_lee_sp','window_pos_lee_vip'),('window_pos_lee_vip','window_pos_lee_sp')],
                            ]
    norm_mean_difference=[]
    for sp_vip_paired_attributte in sp_vip_paired_attributtes: 
        for (sp, vip) in sp_vip_paired_attributte:
            x=greenhouse_climate[filename].iloc[(np.where(greenhouse_climate[filename][sp].isnull()))][vip]
            #print(teams_ghc[sp])
            difference=(greenhouse_climate[filename][sp]-greenhouse_climate[filename][vip]).mean()
            mean_difference=difference/(greenhouse_climate[filename][sp].mean())
            norm_mean_difference.append([sp,vip,mean_difference])

            #print(f'{sp},{vip} difference={norm_mean_difference}')
            #print(x.index)
            for i in x.index:
                #print(i)
                greenhouse_climate[filename].loc[i,sp]= greenhouse_climate[filename].loc[i,vip]#.iloc[i][vip]
                #print(i,teams_ghc.loc[i,sp]) 
            #plt.figure()
            #correlation = greenhouse_climate[filename].corr()[vip][sp]
            #corr1=greenhouse_climate[filename].corr()[vip].sort_values(ascending=False)
            #corr1.plot(kind='bar',title=f'{vip} SP-VIP Correlation',figsize=(18, 6),fontsize=12)
            ##plt.show
            #save_fig(f'{vip}SP-VIP Correlation')
            #print(f"correlation between {vip} and {sp}=",correlation)
            #plt.show
    norm_mean_difference     



    #fill missing values
    for gh_attribute in greenhouse_climate[filename].columns:
        x=greenhouse_climate[filename].iloc[(np.where(greenhouse_climate[filename][gh_attribute].isnull()))][gh_attribute]
        for i in reversed(x.index):
            if i.day+(1)< greenhouse_climate[filename].index[-1].day:
                #print(i+(12*24))
                d=i
                #print(d)

                d = d.replace(day=d.day+1)

                #print(d)
                greenhouse_climate[filename].loc[i,gh_attribute]= greenhouse_climate[filename].loc[d,gh_attribute]
                #print(i+(12*24),teams_ghc.loc[i,gh_attribute])
    for gh_attribute in greenhouse_climate[filename].columns:
        x=greenhouse_climate[filename].iloc[(np.where(greenhouse_climate[filename][gh_attribute].isnull()))][gh_attribute]
        for i in x.index:
            if i.day-(1)>= 0:
                #print(i+(12*24))
                d=i
                #print(d)
                d = d.replace(day=d.day-1)
                #print(d)
                greenhouse_climate[filename].loc[i,gh_attribute]= greenhouse_climate[filename].loc[d,gh_attribute]
                #print(i+(12*24),teams_ghc.loc[i,gh_attribute])

    #print(greenhouse_climate[filename].groupby(greenhouse_climate[filename].index.date).mean())
    
    daily_Avg[filename]=greenhouse_climate[filename].groupby(greenhouse_climate[filename].index.date).mean()
    #daily_Avg[filename]['water_sup_intervals_vip_min']=greenhouse_climate[filename].groupby(greenhouse_climate[filename].index.date).sum()['water_sup_intervals_vip_min']
    daily_Avg[filename]['%time_t']=daily_Avg[filename].index.astype(str)+"_"+filename
    daily_Avg[filename]=daily_Avg[filename].set_index('%time_t')
    #print(daily_Avg[filename].index[-1])
    daily_Avg[filename].drop(daily_Avg[filename].index[-1],axis=0,inplace=True)
    greenhouse_climate[filename]['%time_t']=greenhouse_climate[filename].index.astype(str)+"_"+filename
    greenhouse_climate[filename]=greenhouse_climate[filename].set_index('%time_t')



In [ ]:
import matplotlib.pyplot as plt
for resource in Daily_Resource_Consumption[filename].columns:
    for i in range(len(filenames)):
        Daily_Resource_Consumption[filenames[i]]["2019-12-16 00:00:00" : "2019-12-26 00:00:00"][resource].plot(ylabel=resource,grid=True, marker=".", figsize=(18, 6),legend=True,label=f'{filenames[i]}')
    plt.title(f'green_house_{resource}_time_series_plot for all Teams ')
    plt.legend()
    save_fig(f'green_house_{resource}_time_series_plot ')

    plt.show()

In [ ]:

teams_drc=pd.concat([Daily_Resource_Consumption[filenames[0]],Daily_Resource_Consumption[filenames[1]],
                    Daily_Resource_Consumption[filenames[2]],Daily_Resource_Consumption[filenames[3]],
                    Daily_Resource_Consumption[filenames[4]],Daily_Resource_Consumption[filenames[5]]], axis=0)

teams_ghc=pd.concat([greenhouse_climate[filenames[0]],greenhouse_climate[filenames[1]],
                     greenhouse_climate[filenames[2]],greenhouse_climate[filenames[3]],
                     greenhouse_climate[filenames[4]],greenhouse_climate[filenames[5]]], axis=0)

teams_da=pd.concat([ daily_Avg[filenames[0]],daily_Avg[filenames[1]],
                     daily_Avg[filenames[2]],daily_Avg[filenames[3]],
                     daily_Avg[filenames[4]],daily_Avg[filenames[5]]], axis=0)





In [ ]:
teams_ghc.astype(float).isnull().sum().sum()

In [ ]:
teams_ghc.index

In [ ]:
teams_ghc.hist(figsize=(20,18))

In [ ]:
print(teams_da.shape)
print(teams_drc.shape)

In [ ]:
import matplotlib.pyplot as plt

for num_attribute in teams_drc.columns:
    
    teams_drc["2019-12-16_Automatoes" : "2020-05-29_Automatoes"][num_attribute].plot(ylabel=num_attribute,xlabel='Day Index',grid=True, marker=".", figsize=(18, 6),title=num_attribute)
    
    save_fig(f'dailyConsumptionsof_{num_attribute}_time_series_plot')

    plt.show()

In [ ]:
#teams_da.drop(['assim_vip', 'co2_vip',  'dx_vip',
#       'int_blue_vip', 'int_farred_vip',
#       'int_red_vip', 'int_white_vip',
#        'scr_blck_vip', 
#       'scr_enrg_vip', 't_grow_min_vip', 
#       't_heat_vip', 't_rail_min_vip', 
#       't_ventlee_vip', 't_ventwind_vip',
#       'water_sup_intervals_vip_min',
#       'window_pos_lee_vip'],axis=1,inplace=True)
teams_da.drop(['assim_sp', 'co2_sp',  'dx_sp',
           'int_blue_sp', 'int_farred_sp','scr_blck_sp',
           'int_red_sp', 'int_white_sp','t_grow_min_sp',
           't_heat_sp', 't_rail_min_sp',
           't_vent_sp','water_sup_intervals_sp_min',
           'window_pos_lee_sp','scr_enrg_sp'],axis=1,inplace=True)

In [ ]:
teams_da.shape

In [ ]:
teams_da.index

In [ ]:
teams_drc.index

In [ ]:
#teams_drc.drop('%Time_t',axis=1,inplace=True)
#teams_ghc.drop('%time_t',axis=1,inplace=True)

In [ ]:
teams_drc.isnull().sum()

In [ ]:
teams_ghc.shape

In [ ]:
teams_ghc.isnull().sum()

In [ ]:
#teams_ghc.drop(columns=[  'AssimLight', 'BlackScr', 'CO2air', 'EC_drain_PC', 'EnScr', 'HumDef',
#       'PipeGrow', 'PipeLow', 'Rhair', 'Tair', 'Tot_PAR', 'Tot_PAR_Lamps',
#       'VentLee', 'Ventwind', 'co2_dos', 'Cum_irr','water_sup'],inplace=True)

features = ['assim_sp', 'assim_vip', 'co2_dos', 'co2_sp',
       'co2_vip', 'dx_sp', 'dx_vip', 'int_blue_sp', 'int_blue_vip',
       'int_farred_sp', 'int_farred_vip', 'int_red_sp', 'int_red_vip',
       'int_white_sp', 'int_white_vip', 'scr_blck_sp',
       'scr_blck_vip', 'scr_enrg_sp', 'scr_enrg_vip', 't_grow_min_sp',
       't_grow_min_vip', 't_heat_sp', 't_heat_vip', 't_rail_min_sp',
       't_rail_min_vip', 't_vent_sp', 't_ventlee_vip', 't_ventwind_vip',
       'water_sup_intervals_sp_min', 'window_pos_lee_sp', 'window_pos_lee_vip',
       'water_sup_intervals_vip_min']

In [ ]:
teams_ghc.columns

In [ ]:
teams_ghc.index.shape

In [ ]:
#teams_drc['Irr']=teams_drc['Irr']-teams_drc['Drain']
#teams_drc.drop(['Drain'],axis=1,inplace=True)
teams_drc.shape

In [ ]:
for i in range(len(filenames)):

    greenhouse_climate[filenames[i]].drop(['assim_sp', 'co2_sp',  'dx_sp',
           'int_blue_sp', 'int_farred_sp','scr_blck_sp',
           'int_red_sp', 'int_white_sp','t_grow_min_sp',
           't_heat_sp', 't_rail_min_sp', 
           't_vent_sp','water_sup_intervals_sp_min',
           'window_pos_lee_sp','scr_enrg_sp'],axis=1,inplace=True)
    daily_Avg[filenames[i]].drop(['assim_sp', 'co2_sp',  'dx_sp',
           'int_blue_sp', 'int_farred_sp','scr_blck_sp',
           'int_red_sp', 'int_white_sp','t_grow_min_sp',
           't_heat_sp', 't_rail_min_sp',
           't_vent_sp','water_sup_intervals_sp_min',
           'window_pos_lee_sp','scr_enrg_sp'],axis=1,inplace=True)

In [ ]:
teams_drc.set_index(teams_da.index,inplace=True)

In [ ]:
teams_drc.index

In [ ]:
daily_data=pd.concat([ teams_da,teams_drc], axis=1)
daily_data.head()

In [ ]:
daily_data.shape


In [ ]:
attributes=['Heat_cons'	,'ElecHigh',	'ElecLow',	'CO2_cons',	'Irr','Drain']
for  attribute in attributes:
    correlation=daily_data.corr()[attribute].sort_values(ascending=False).drop(teams_drc.columns)
    plt.figure()
    correlation.plot(kind='bar',title=f'{attribute} Correlation with features',figsize=(18, 6),fontsize=12)
    plt.show
    save_fig(f'{attribute}CorrelationwithFeatures')
    plt.show

In [ ]:
# Required Libraries
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
teams_data=dict()
corr=dict()
team_corr=dict()
filenames = ['AICU', 'Automatoes',
             'Digilog', 'IUACAAS', 
             'Reference', 'TheAutomators'
             ]
for i in range(len(filenames)):

    merged_data=pd.concat([ daily_Avg[filenames[i]],Daily_Resource_Consumption[filenames[i]]], axis=1)
    
    correlation_matrix = merged_data.corr()
    
    outputs=merged_data.columns[-6:]
    
    for output in outputs:
        corr[output]=merged_data[merged_data.columns].corr()[output].sort_values(ascending=False)
        corr[output].drop(outputs,axis=0,inplace=True)
        corr[output].plot(kind='bar',title=f'{filenames[i]} {output} Correlation with SP',figsize=(18, 6),fontsize=12)
        plt.show()
        save_fig(f'{filenames[i]} {output} Correlation with SP')
        
    team_corr[filenames[i]]=corr
    # Plotting the correlation matrix
    #plt.figure(figsize=(15, 10))
    #sns.heatmap(correlation_matrix, annot=False, cmap='coolwarm')
    #plt.title('Correlation Matrix Between SP Features and Crop Parameters')
    #plt.show()
    
    #print('control_features',merged_data.columns[:-3])
    control_features = merged_data.columns[:-6]
    
    #print('output',merged_data.columns[-5:-2])
    
    #for control_feature in control_features:
    #    plt.figure(figsize=(15, 6))
    #    sns.lineplot(data=merged_data[control_feature], palette='tab11',legend=True)           
    #    sns.lineplot(data=merged_data[outputs], palette='tab10')
#
    #    plt.title(f'Time Series of {control_feature} and CropParameters for {filenames[i]}')
    #    plt.xlabel('Date')
    #    plt.legend(loc='right')
    #    plt.show()
    #for control_feature in control_features:
    #    plt.figure(figsize=(15, 6))
    #    for output in outputs:
    #        sns.lineplot(x=merged_data[control_feature],y=merged_data[output], palette='tab10')
    #        #sns.lineplot(data=teams_ghc_weekly[output], palette='tab10')
    #plt.title(f'{control_feature} with CropParameters {filenames[i]}')
    #plt.xlabel(control_feature)
    #plt.ylabel('CropParameters')
    #plt.legend(loc='best')
    #plt.show()
    
    # Step 8: Effect of Specific Control Setpoints on Crop Parameters
    for control_feature in control_features:
        for output in outputs:
            plt.figure(figsize=(10, 6))
            sns.scatterplot(data=merged_data, x=control_feature, y=output)
            sns.regplot(data=merged_data, x=control_feature, y=output, scatter=False, color='red')
            plt.title(f'Effect of {control_feature} on {output} for {filenames[i]}')
            plt.show()

    teams_data[filenames[i]]=merged_data

In [ ]:
import plotly.express as px
#add 'AssimLight', 'CO2air', 'Cum_irr','HumDef', 'Rhair', 'Tair', 'Tot_PAR','Tot_PAR_Lamps', 'co2_dos',
features=[       'co2_vip', 'dx_vip', 'int_blue_vip', 'int_farred_vip', 'int_red_vip',
       'int_white_vip', 'pH_drain_PC', 'scr_blck_vip','t_heat_vip',
       'water_sup_intervals_vip_min']
# Step 10: Advanced Visualizations
for feature in features:
    for output in outputs:
        # Interactive Scatter Plot with Plotly
        fig = px.scatter(daily_data, x=feature, y=output, color=[filenames[i] for x in range(166) for i in range(len(filenames))])       
        fig.update_layout(title=f'Interactive Scatter Plot: {feature} vs. {output}')
        fig.show()

In [ ]:
import plotly.express as px
import seaborn as sns
# Violin Plot to Compare Distributions
for output in outputs:
    plt.figure(figsize=(10, 6))
    sns.violinplot(data=daily_data, x=[filenames[i] for x in range(166)  for i in range(len(filenames))], y=output)
    plt.title('Violin Plot for Stem Elongation Across Teams')
    plt.show()



In [ ]:
for i in range(len(filenames)):
        Daily_Resource_Consumption[filenames[i]]['%time'] = pd.to_datetime(Daily_Resource_Consumption[filenames[i]].index.str.split('_').str[0])
        Daily_Resource_Consumption[filenames[i]].set_index('%time', inplace=True)  # Set datetime as the index


In [ ]:
for feature in Daily_Resource_Consumption[filename].columns:
    fig = plt.figure(figsize=(10, 7))
    ax = fig.gca()
    for i in range(len(filenames)):
        
        davg = Daily_Resource_Consumption[filenames[i]][feature]#.resample('h').mean()  # compute the mean for each hour
        #weekly_avg = Weekly_Crop_Parameters[filenames[i]][output].resample('W').mean()
        #print(dayly_avg)
        period=slice("2019-12-16 00:00:00", "2020-05-30 00:00:00")
        rolling_average_daily = davg[period].rolling(window=30).mean()
        #rolling_average_weekly = weekly_avg[period].rolling(window=4).mean()

        rolling_average_daily.plot(grid=True,legend=True,label=f'{filenames[i]}')
    ax.set_xlabel(xlabel='%time')
    ax.set_ylabel(ylabel=f'{feature}')
    ax.set_title(f'{feature} time series of Teams')

    plt.show()

#add the legend for each line of plot which represent the file_name

In [ ]:
# import the necessary packages
import json   
from math import floor
from keras.preprocessing.image import ImageDataGenerator
from keras.applications import vgg19
from keras.layers import AveragePooling2D,Dropout,Flatten,Dense,Activation,Input,MaxPooling2D
from keras.models import Model
from keras.optimizers import Adam
from keras.applications.vgg19 import preprocess_input
from keras.utils import to_categorical
from sklearn.preprocessing import OneHotEncoder,LabelEncoder,StandardScaler,MinMaxScaler
from sklearn.preprocessing import LabelBinarizer

from keras.callbacks import LearningRateScheduler
from keras.callbacks import EarlyStopping,ModelCheckpoint
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report,confusion_matrix,multilabel_confusion_matrix
from imutils import paths
import matplotlib.pyplot as plt
import numpy as np
import argparse
import os
import cv2
from sklearn.metrics import mean_squared_error,r2_score


In [ ]:
daily_data.duplicated().sum()

In [ ]:
labels = daily_data[attributes]
data = daily_data.drop(attributes,axis=1)

np.save('daily_data.npy', data)
np.save('daily_labels.npy', labels)

print('data',data.shape)
print('labels',labels.shape)

#data = np.load("data.npy")
#labels = np.load("labels.npy")

#testY=ss_y.transform(testY)

#testY=pd.DataFrame(testY,columns=[labels.columns])


#splitting the data into test and train
(trainX, testX, trainY, testY) = train_test_split(data, labels,shuffle=True,
	test_size=0.2, random_state=42)

#(trainX, validX, trainY, validY) = train_test_split(trainX, trainY,shuffle=True,
#	test_size=0.2, random_state=42)

ss_x=MinMaxScaler()
trainX=ss_x.fit_transform(trainX)
testX=ss_x.transform(testX)

ss_y=MinMaxScaler()
trainY=ss_y.fit_transform(trainY)
testY=ss_y.transform(testY)

print('trainX',trainX.shape)
print('trainY',trainY.shape)
print('testX',testX.shape)
print('testY',testY.shape)
#print('validX',validX.shape)
#print('validY',validY.shape)


#SS = StandardScaler()
#
#trainX = SS.fit_transform(trainX)
#validX = SS.transform(validX)
#testX= SS.transform(testX)


np.save('trainX_DailyResourceConsumptionRegression.npy', trainX)
np.save('trainY_DailyResourceConsumptionRegression.npy', trainY)

np.save('testX_DailyResourceConsumptionRegression.npy', testX)
np.save('testY_DailyResourceConsumptionRegression.npy', testY)

#np.save('validX_DailyResourceConsumptionRegression.npy', validX)
#np.save('validY_DailyResourceConsumptionRegression.npy', validY)



#trainX = np.load("trainX_DHRegression.npy")
#trainY = np.load("trainY_DHRegression.npy")
#testX = np.load("testX_DHRegression.npy")
#testY = np.load("testY_DHRegression.npy")
#validX = np.load("validX_DHRegression.npy")
#validY = np.load("validY_DHRegression.npy")

trainX=pd.DataFrame(trainX,columns=data.columns)
testX=pd.DataFrame(testX,columns=data.columns)
trainY=pd.DataFrame(trainY,columns=labels.columns)
testY=pd.DataFrame(testY,columns=labels.columns)


print('trainX',trainX.min(),trainX.max())
print('trainY',trainY.min(),trainY.max())
print('testX',testX.min(),testX.max())
print('testY',testY.min(),testY.max())







In [ ]:
col=Daily_Resource_Consumption[filenames[0]].columns
idx=Daily_Resource_Consumption[filenames[0]].index

In [ ]:
weekly_resourceConsumption=dict()

In [ ]:
Daily_Resource_Consumption[filenames[0]]=ss_y.transform(Daily_Resource_Consumption[filenames[0]])
Daily_Resource_Consumption[filenames[1]]=ss_y.transform(Daily_Resource_Consumption[filenames[1]])
Daily_Resource_Consumption[filenames[2]]=ss_y.transform(Daily_Resource_Consumption[filenames[2]])
Daily_Resource_Consumption[filenames[3]]=ss_y.transform(Daily_Resource_Consumption[filenames[3]])
Daily_Resource_Consumption[filenames[4]]=ss_y.transform(Daily_Resource_Consumption[filenames[4]])
Daily_Resource_Consumption[filenames[5]]=ss_y.transform(Daily_Resource_Consumption[filenames[5]])

Daily_Resource_Consumption[filenames[0]]=pd.DataFrame(Daily_Resource_Consumption[filenames[0]],columns=col,index=idx)
Daily_Resource_Consumption[filenames[1]]=pd.DataFrame(Daily_Resource_Consumption[filenames[1]],columns=col,index=idx)
Daily_Resource_Consumption[filenames[2]]=pd.DataFrame(Daily_Resource_Consumption[filenames[2]],columns=col,index=idx)
Daily_Resource_Consumption[filenames[3]]=pd.DataFrame(Daily_Resource_Consumption[filenames[3]],columns=col,index=idx)
Daily_Resource_Consumption[filenames[4]]=pd.DataFrame(Daily_Resource_Consumption[filenames[4]],columns=col,index=idx)
Daily_Resource_Consumption[filenames[5]]=pd.DataFrame(Daily_Resource_Consumption[filenames[5]],columns=col,index=idx)




In [ ]:
weekly_resourceConsumption[filenames[0]]=Daily_Resource_Consumption[filenames[0]].resample('W').sum()
weekly_resourceConsumption[filenames[1]]=Daily_Resource_Consumption[filenames[1]].resample('W').sum()
weekly_resourceConsumption[filenames[2]]=Daily_Resource_Consumption[filenames[2]].resample('W').sum()
weekly_resourceConsumption[filenames[3]]=Daily_Resource_Consumption[filenames[3]].resample('W').sum()
weekly_resourceConsumption[filenames[4]]=Daily_Resource_Consumption[filenames[4]].resample('W').sum()
weekly_resourceConsumption[filenames[5]]=Daily_Resource_Consumption[filenames[5]].resample('W').sum()

In [ ]:
weekly_resourceConsumption[filenames[0]]=pd.DataFrame(weekly_resourceConsumption[filenames[0]],columns=col)
weekly_resourceConsumption[filenames[1]]=pd.DataFrame(weekly_resourceConsumption[filenames[1]],columns=col)
weekly_resourceConsumption[filenames[2]]=pd.DataFrame(weekly_resourceConsumption[filenames[2]],columns=col)
weekly_resourceConsumption[filenames[3]]=pd.DataFrame(weekly_resourceConsumption[filenames[3]],columns=col)
weekly_resourceConsumption[filenames[4]]=pd.DataFrame(weekly_resourceConsumption[filenames[4]],columns=col)
weekly_resourceConsumption[filenames[5]]=pd.DataFrame(weekly_resourceConsumption[filenames[5]],columns=col)

In [ ]:
weekly_resourceConsumption[filenames[0]].index

In [ ]:
print(weekly_resourceConsumption[filenames[0]].columns)

In [ ]:
weekly_resourceConsumption[filenames[0]].to_csv(f'{filenames[0]}resource_consumption.csv')
weekly_resourceConsumption[filenames[1]].to_csv(f'{filenames[1]}resource_consumption.csv')
weekly_resourceConsumption[filenames[2]].to_csv(f'{filenames[2]}resource_consumption.csv')
weekly_resourceConsumption[filenames[3]].to_csv(f'{filenames[3]}resource_consumption.csv')
weekly_resourceConsumption[filenames[4]].to_csv(f'{filenames[4]}resource_consumption.csv')
weekly_resourceConsumption[filenames[5]].to_csv(f'{filenames[5]}resource_consumption.csv')

In [ ]:
from sklearn.pipeline import Pipeline,make_pipeline
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import LinearSVR 
from sklearn.svm import SVR
from sklearn.model_selection import cross_val_score,cross_val_predict
import numpy as np
from sklearn.compose import ColumnTransformer,make_column_selector
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler,OneHotEncoder
from sklearn.ensemble import GradientBoostingRegressor,AdaBoostRegressor
from sklearn.linear_model import Lasso,Ridge,ElasticNet
from sklearn.preprocessing import FunctionTransformer


In [ ]:
#Multioutput Regression

models={#'LinearRegression':LinearRegression(fit_intercept=True,positive=True,n_jobs=-1),
        'RandomForest':RandomForestRegressor( random_state = 42,
                                            n_estimators=80,criterion='squared_error',
                                            max_depth= None, min_samples_split = 2, min_samples_leaf= 1, min_weight_fraction_leaf = 0,
                                            max_features='sqrt',max_leaf_nodes = None, min_impurity_decrease = 0, bootstrap=True,
                                            oob_score  = False, n_jobs = None, verbose = 0, warm_start = True,
                                            ccp_alpha = 0, max_samples=None
                                             ),
        'GradientBoosting':GradientBoostingRegressor(random_state=42),
        'AdaBoost':AdaBoostRegressor(random_state=42),
        'KNeighborsRegressor':KNeighborsRegressor(n_neighbors= 7, weights='distance', algorithm='ball_tree', leaf_size = 50,
                                                   p= 4, metric = "minkowski",metric_params = None, n_jobs=-1),
        'DecisionTree':DecisionTreeRegressor(random_state=42),
        #'LinearSVR':LinearSVR(max_iter=50000,random_state=42),
        'SVR':  SVR(max_iter=50000)
        }

num_attributes=data.columns
cat_attributes=[]
outputs=labels.columns
num_pipline=make_pipeline(SimpleImputer())
cat_pipeline=make_pipeline(OneHotEncoder())

preprocessing=ColumnTransformer([('num', num_pipline, num_attributes),
                                ('cat',cat_pipeline,cat_attributes),
                                ])



from sklearn.metrics import mean_squared_error,r2_score
from sklearn.model_selection import cross_val_score,cross_val_predict,cross_validate

outputs=teams_drc.columns
train_results=dict()
train_results[outputs[0]]=[] 
train_results[outputs[1]]=[]
train_results[outputs[2]]=[]
train_results[outputs[3]]=[]
train_results[outputs[4]]=[]
train_results[outputs[5]]=[]
test_results=dict()
test_results[outputs[0]]=[] 
test_results[outputs[1]]=[]
test_results[outputs[2]]=[]
test_results[outputs[3]]=[]
test_results[outputs[4]]=[]
test_results[outputs[5]]=[]

conf_interv_results=dict()
conf_interv_results[outputs[0]]=[]
conf_interv_results[outputs[1]]=[]
conf_interv_results[outputs[2]]=[]
conf_interv_results[outputs[3]]=[]
conf_interv_results[outputs[4]]=[]
conf_interv_results[outputs[5]]=[]

train_rmse=dict()
train_r2=dict()
train_mse=dict()
test_rmse=dict()
test_r2=dict()
test_mse=dict()
c1=dict()
c2=dict()
from sklearn.multioutput import MultiOutputRegressor
from sklearn.multioutput import RegressorChain
import joblib

for key in models.keys():

    full_pipeline = Pipeline([
        ('preprocessing', preprocessing),
        ('models',RegressorChain(models[key],order=[0,1,2,3,4,5],random_state=42)),
        #('models',MultiOutputRegressor(models[key],n_jobs=-1)),
    ])
    #train
    score=cross_validate(full_pipeline, trainX, trainY,scoring='neg_root_mean_squared_error', cv=10)
    full_pipeline.fit(trainX, trainY)
    train_prediction=cross_val_predict(full_pipeline,trainX,trainY,cv=3)
    
    #train_prediction=min_max_scaler.inverse_transform(train_prediction)
    test_prediction=cross_val_predict(full_pipeline,testX,testY,cv=3)
    #train_prediction=min_max_scaler.inverse_transform(test_prediction)
    joblib.dump(full_pipeline,f'{key}DailyResourceConsumptionEstimator.pkl')    
    #loaded_model=joblib.load(f'{key}DailyResourceConsumptionEstimator.pkl')    
    #new_data=testX
    #predictions=loaded_model.predict(new_data)
    #print(predictions,testY)    
    for i,out in zip(range(0,len(labels.columns),1),labels.columns):
        #train scores:
        train_rmse[out]=mean_squared_error(y_pred=train_prediction[:,i],y_true=trainY[out])/labels[out].mean()
        train_mse[out]=mean_squared_error(y_pred=train_prediction[:,i],y_true=trainY[out])/labels[out].mean()
        train_r2[out]=r2_score(y_true=trainY[out],y_pred=train_prediction[:,i])
        #train result for each output
        train_results[out].append((key,train_rmse[out],train_mse[out],train_r2[out]))
       
        #test scores:
        test_rmse[out]=mean_squared_error(y_pred=test_prediction[:,i],y_true=testY[out])/labels[out].mean()
        test_mse[out]=mean_squared_error(y_pred=test_prediction[:,i],y_true=testY[out])/labels[out].mean()
        test_r2[out]=r2_score(y_true=testY[out],y_pred=test_prediction[:,i])
        #test result for each output
        test_results[out].append((key,test_rmse[out],test_mse[out],test_r2[out]))
        
        from scipy import stats
        #confidence_interval:
        confidence = 0.95
        #print(test_prediction[:,i].shape,testY[out].shape)
        squared_errors = (test_prediction[:,i] - testY[out]) ** 2
        zscore = stats.norm.ppf((1 + confidence) / 2)
        zmargin = zscore * squared_errors.std(ddof=1) / np.sqrt(len(squared_errors))
        c1[out] = np.sqrt(squared_errors.mean() - zmargin)/labels[out].mean()
        c2[out]= np.sqrt(squared_errors.mean() + zmargin)/labels[out].mean()
        #confidence_intervals for each output:
        conf_interv_results[out].append((key,test_rmse[out],c1[out],c2[out],c2[out]-c1[out]))
        
#        # Plot predicted vs actual
#        plt.figure
#        plt.style.use("ggplot")
#        plt.scatter(testY[out], test_prediction[:,i],color='steelblue')
#        plt.xlabel(f'Actual {out}')
#        plt.ylabel(f'Predicted {out}')
#        plt.title(f'predicted vs actual {out} \nplot for {key}',fontsize=12)
#        # overlay the regression line
#        z = np.polyfit(testY[out], test_prediction[:,i], 1)
#        p = np.poly1d(z)
#        plt.plot(testY[out],p(testY[out]),color='red')
#        #plt.grid()
#        save_fig(f'predicted vs actual {out} plot for {key}')
#        plt.show()



        from scipy import stats
        #confidence_interval:
        confidence = 0.95
        squared_errors = (test_prediction[:,i] - testY[out]) ** 2
        zscore = stats.norm.ppf((1 + confidence) / 2)
        zmargin = zscore * squared_errors.std(ddof=1) / np.sqrt(len(squared_errors))
        c1[out] = np.sqrt(squared_errors.mean() - zmargin)/labels[out].mean()
        c2[out] = np.sqrt(squared_errors.mean() + zmargin)/labels[out].mean()
        #confidence_intervals for each output:
        conf_interv_results[out].append((key,test_rmse[out],c1[out],c2[out],c2[out]-c1[out]))
        

    
print(train_results[outputs[0]],'\n',train_results[outputs[1]],'\n',train_results[outputs[2]],
      '\n',train_results[outputs[3]],'\n',train_results[outputs[4]],'\n',train_results[outputs[5]],'\n')
print(test_results[outputs[0]],'\n',test_results[outputs[1]],'\n',test_results[outputs[2]],'\n',
      test_results[outputs[3]],'\n',test_results[outputs[4]],'\n',test_results[outputs[5]],'\n')


In [ ]:
#show training results table:
best_trained_model_idx=dict()
train_result=dict()

for out in outputs:
    print(f'best models train {out} scores:',train_results[out])
    best_trained_model_idx[out]=np.array(train_results[out])[:,1].argmin()


    #print('best models test scores:',test_results)
    #best_tested_model_idx=np.array(test_results)[:,1].argmax()

    print('best trained model:',train_results[out][best_trained_model_idx[out]][0],
                            train_results[out][best_trained_model_idx[out]][1],'\n')


    #print('best tested model:',test_results[best_tested_model_idx][0],test_results[best_tested_model_idx][1])
    train_result[out]=pd.DataFrame(train_results[out],
               columns=['Model','RMSE','MSE','R2_Score']).sort_values(by='RMSE')

    #dfi.export(train_result, "ModelsTrainingResultsTable.png")
    print(train_result[out])


In [ ]:
print(outputs[0],'\n')
train_result[outputs[0]]

In [ ]:
print(outputs[1],'\n')
train_result[outputs[1]]

In [ ]:
print(outputs[2],'\n')
train_result[outputs[2]]

In [ ]:
print(outputs[3],'\n')
train_result[outputs[3]]

In [ ]:
print(outputs[4],'\n')
train_result[outputs[4]]

In [ ]:
print(outputs[5],'train result\n')
train_result[outputs[5]]

In [ ]:
#show training results table:
best_trained_model_idx=dict()
test_result=dict()

for out in outputs:
    print(f'best models train {out} scores:',test_results[out])
    best_trained_model_idx[out]=np.array(test_results[out])[:,1].argmin()


    #print('best models test scores:',test_results)
    #best_tested_model_idx=np.array(test_results)[:,1].argmax()

    print('best trained model:',test_results[out][best_trained_model_idx[out]][0],
                            test_results[out][best_trained_model_idx[out]][1],'\n')


    #print('best tested model:',test_results[best_tested_model_idx][0],test_results[best_tested_model_idx][1])
    test_result[out]=pd.DataFrame(test_results[out],
               columns=['Model','RMSE','MSE','R2_Score']).sort_values(by='RMSE')

    #dfi.export(test_result, "ModelsTrainingResultsTable.png")
    print(test_result[out])


In [ ]:
print(outputs[0],'test result\n')
test_result[outputs[0]]

In [ ]:
print(outputs[1],'test result','\n')
test_result[outputs[1]]

In [ ]:
print(outputs[2],'test result\n')
test_result[outputs[2]]

In [ ]:
print(outputs[3],'test result\n')
test_result[outputs[3]]

In [ ]:
print(outputs[4],'test result\n')
test_result[outputs[4]]

In [ ]:
print(outputs[5],'test result\n')
test_result[outputs[5]]

In [ ]:
conf_interv_result=dict()
for out in outputs:

    print(conf_interv_results[out],'\n')

    
    conf_interv_result[out]=pd.DataFrame(conf_interv_results[out],
               columns=['Model','RMSE','C1','C2','Confidence Interval']).sort_values(by='RMSE',ignore_index=True)

    
    conf_interv_result[out]

In [ ]:
print(outputs[0],'\n')
conf_interv_result[outputs[0]]

In [ ]:
print(outputs[1],'\n')
conf_interv_result[outputs[1]]

In [ ]:
print(outputs[2],'\n')
conf_interv_result[outputs[2]]

In [ ]:
print(outputs[3],'\n')
conf_interv_result[outputs[3]]

In [ ]:
print(outputs[4],'\n')
conf_interv_result[outputs[4]]

In [ ]:
print(outputs[5],'\n')
conf_interv_result[outputs[5]]

In [ ]:
for out in outputs:
    plt.figure(figsize=(6,6))
    plt.xticks(range(len(conf_interv_result[out])),list(conf_interv_result[out]['Model']),
                rotation = 90, ha="right")
    plt.title(f'RMSE Confidence Intevals of {out}\n for Classical ML Models',fontsize=12)
    for C1,C2,b in zip(conf_interv_result[out]['C1'],conf_interv_result[out]['C2'],
                        range(len(conf_interv_result[out]))):
        color='#2187bb'
        horizontal_line_width=0.25
        left = b - horizontal_line_width / 4
        right = b + horizontal_line_width / 4
        #plt.plot((b,b),(C1,C2),'ro-',color='orange')
        plt.plot([b, b], [C2, C1], color=color)
        plt.plot([left, right], [C2, C2], color=color)
        plt.plot([left, right], [C1, C1], color=color)
        plt.plot(b, conf_interv_result[out]['RMSE'][b], 'o', color='#f44336')
    save_fig(f'RMSE_Confidence_Intevals_{out}_ML_Models')

    #plt.xticks(range(len(confidence_interval_results)),list(confidence_interval_results['Model']))
    plt.show()

In [ ]:
#for pred_out,i in zip(outputs,range(len(outputs))):
#    plt.style.use("ggplot")
#    plt.figure()
#    plt.plot(test_prediction[:,i][15:50],label='predected', marker=".")
#    plt.plot(testY[pred_out].values[15:50],label='Actual',marker=".")
#
#    plt.title(f"Predicted Vs. Actual of {pred_out} \nsamples of {Chain Regressorgressor}",fontsize=12)
#    plt.xlabel("samples")
#    plt.ylabel(f"{pred_out}")
#    plt.legend(loc="lower left")
#    save_fig(f"Predicted Vs. Actual of {pred_out} in discrete time series points of Chain Regressor")
#
#    plt.show()

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV,RandomizedSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.compose import TransformedTargetRegressor
from sklearn.metrics import mean_squared_error,r2_score
from scipy.stats import randint
from sklearn.multioutput import RegressorChain



outputs=teams_drc.columns
train_results=dict()
train_results[outputs[0]]=[] 
train_results[outputs[1]]=[]
train_results[outputs[2]]=[]
train_results[outputs[3]]=[]
train_results[outputs[4]]=[]
train_results[outputs[5]]=[]
test_results=dict()
test_results[outputs[0]]=[] 
test_results[outputs[1]]=[]
test_results[outputs[2]]=[]
test_results[outputs[3]]=[]
test_results[outputs[4]]=[]
test_results[outputs[5]]=[]

conf_interv_results=dict()
conf_interv_results[outputs[0]]=[]
conf_interv_results[outputs[1]]=[]
conf_interv_results[outputs[2]]=[]
conf_interv_results[outputs[3]]=[]
conf_interv_results[outputs[4]]=[]
conf_interv_results[outputs[5]]=[]

train_rmse=dict()
train_r2=dict()
train_mse=dict()
test_rmse=dict()
test_r2=dict()
test_mse=dict()
c1=dict()
c2=dict()
from sklearn.multioutput import MultiOutputRegressor
from sklearn.multioutput import RegressorChain



estimator=RandomForestRegressor(n_estimators= 100,criterion='squared_error',
    max_depth= None,min_samples_split= 2,min_samples_leaf= 1,min_weight_fraction_leaf= 0,max_features='sqrt',
    max_leaf_nodes= None,min_impurity_decrease= 0,bootstrap= True,oob_score=False,n_jobs= -1,
    ccp_alpha=0,max_samples= None,random_state=42)

best_models=dict()

best_models['Chain_Regressor']=RegressorChain(base_estimator=estimator,order=[0,1,2,3,4,5],random_state=42)
full_pipeline = Pipeline([
    ('preprocessing', preprocessing),
    ('Chain_Regressor',best_models['Chain_Regressor'] )
                        ])
       
param_grid=[{ 
                'Chain_Regressor__base_estimator__max_depth':[2,6,None,8],
                'Chain_Regressor__base_estimator__n_estimators':[100,120,150,200,300,500],
                'Chain_Regressor__base_estimator__criterion':['squared_error','friedman_mse'],
                'Chain_Regressor__base_estimator__min_samples_split':[2,10,8],
                'Chain_Regressor__base_estimator__max_samples':[None,100,20,80],
                'Chain_Regressor__base_estimator__max_features':['sqrt','log2'],
                'Chain_Regressor__base_estimator__ccp_alpha':[0,.1,.01,10,1],              
            }]
grid_search=GridSearchCV(full_pipeline,param_grid,cv=10,scoring='neg_root_mean_squared_error',verbose=2)


In [ ]:
#grid_search.fit(trainX,trainY)


In [ ]:
#train_prediction=cross_val_predict(grid_search.best_estimator_,trainX,trainY,cv=3)
#test_prediction=cross_val_predict(grid_search.best_estimator_,testX,testY,cv=3)


In [ ]:


#for i,out in zip(range(0,len(labels.columns),1),labels.columns):
#    #train scores:
#    train_rmse[out]=mean_squared_error(y_pred=train_prediction[:,i],y_true=trainY[out])/labels[out].mean()
#    train_mse[out]=mean_squared_error(y_pred=train_prediction[:,i],y_true=trainY[out])/labels[out].mean()
#    train_r2[out]=r2_score(y_true=trainY[out],y_pred=train_prediction[:,i])
#    #train result for each output
#    train_results[out].append(('Chain_Regressor',train_rmse[out],train_mse[out],train_r2[out]))
#   
#    #test scores:
#    test_rmse[out]=mean_squared_error(y_pred=test_prediction[:,i],y_true=testY[out])/labels[out].mean()
#    test_mse[out]=mean_squared_error(y_pred=test_prediction[:,i],y_true=testY[out])/labels[out].mean()
#    test_r2[out]=r2_score(y_true=testY[out],y_pred=test_prediction[:,i])
#    #test result for each output
#    test_results[out].append(('Chain_Regressor',test_rmse[out],test_mse[out],test_r2[out]))
#    
#    # Plot predicted vs actual
#    plt.style.use("ggplot")
#    plt.scatter(testY[out], test_prediction[:,i],color='steelblue')
#    plt.xlabel(f'Actual {out}')
#    plt.ylabel(f'Predicted {out}')
#    plt.title(f'predicted vs actual {out} \nplot for Chain Regressor',fontsize=12)
#    # overlay the regression line
#    z = np.polyfit(testY[out], test_prediction[:,i], 1)
#    p = np.poly1d(z)
#    plt.plot(testY[out],p(testY[out]),color='red')
#    #plt.grid()
#    save_fig(f'predicted vs actual {out} plot for Chain Regressor')
#    plt.show()
#
#
#
#    from scipy import stats
#    #confidence_interval:
#    confidence = 0.95
#    squared_errors = (test_prediction[:,i] - testY[out]) ** 2
#    zscore = stats.norm.ppf((1 + confidence) / 2)
#    zmargin = zscore * squared_errors.std(ddof=1) / np.sqrt(len(squared_errors))
#    c1[out] = np.sqrt(squared_errors.mean() - zmargin)/labels[out].mean()
#    c2[out] = np.sqrt(squared_errors.mean() + zmargin)/labels[out].mean()
#    #confidence_intervals for each output:
#    conf_interv_results[out].append(('Chain_Regressor',test_rmse[out],c1[out],c2[out],c2[out]-c1[out]))
#    
#print(train_results[outputs[0]],'\n',train_results[outputs[1]],'\n',train_results[outputs[2]],'\n',
#      train_results[outputs[3]],'\n',train_results[outputs[4]],'\n',train_results[outputs[5]],'\n')
#print(test_results[outputs[0]],'\n',test_results[outputs[1]],'\n',test_results[outputs[2]],'\n',
#      test_results[outputs[3]],'\n',test_results[outputs[4]],'\n',test_results[outputs[5]],'\n')
#print(conf_interv_results[outputs[0]],'\n',conf_interv_results[outputs[1]],'\n',conf_interv_results[outputs[2]],'\n',
#      conf_interv_results[outputs[3]],'\n',conf_interv_results[outputs[4]],'\n',conf_interv_results[outputs[5]],'\n')
#

In [ ]:
#for pred_out,i in zip(outputs,range(len(outputs))):
#    plt.style.use("ggplot")
#    plt.figure()
#    plt.plot(test_prediction[:,i][15:50],label='predected', marker=".")
#    plt.plot(testY[pred_out].values[15:50],label='Actual',marker=".")
#
#    plt.title(f"Predicted Vs. Actual of {pred_out} \nsamples of Chain Regressor",fontsize=12)
#    plt.xlabel("samples")
#    plt.ylabel(f"{pred_out}")
#    plt.legend(loc="lower left")
#    save_fig(f"Predicted Vs. Actual of {pred_out} in discrete time series points of Chain Regressor")
#
#    plt.show()

In [ ]:
#best_params=grid_search.best_params_
##best_params['RandomForestRegressor__base_estimator']=grid_search.best_estimator_['RandomForestRegressor__base_estimator'].n_estimators_
#GBR_best_param=pd.DataFrame.from_dict(grid_search.best_params_,orient='index',columns=['best value'])
#GBR_best_param['best value']

In [ ]:
##show training results table:
#best_trained_model_idx=dict()
#test_result=dict()
#
#for out in outputs:
#    print(f'best models train {out} scores:',test_results[out])
#    best_trained_model_idx[out]=np.array(test_results[out])[:,1].argmin()
#
#
#    #print('best models test scores:',test_results)
#    #best_tested_model_idx=np.array(test_results)[:,1].argmax()
#
#    print('best trained model:',test_results[out][best_trained_model_idx[out]][0],
#                            test_results[out][best_trained_model_idx[out]][1],'\n')
#
#
#    #print('best tested model:',test_results[best_tested_model_idx][0],test_results[best_tested_model_idx][1])
#    test_result[out]=pd.DataFrame(test_results[out],
#               columns=['Model','RMSE','MSE','R2_Score']).sort_values(by='RMSE')
#
#    #dfi.export(test_result, "ModelsTrainingResultsTable.png")
#    print(test_result[out])
#

In [ ]:
#print(outputs[0],'\n')
#test_result[outputs[0]]

In [ ]:
#print(outputs[1],'\n')
#test_result[outputs[1]]

In [ ]:
#print(outputs[2],'\n')
#test_result[outputs[2]]

In [ ]:
#print(outputs[3],'\n')
#test_result[outputs[3]]

In [ ]:
#print(outputs[4],'\n')
#test_result[outputs[4]]

In [ ]:
#print(outputs[5],'\n')
#test_result[outputs[5]]

In [ ]:
##grid_search.best_estimator_.save('DailyResourceConsumptionEstimator.pkl')
#
#import joblib
#
#joblib.dump(grid_search.best_estimator_,'DailyResourceConsumptionEstimator.pkl')
#
#loaded_model=joblib.load('DailyResourceConsumptionEstimator.pkl')
#
#new_data=testX
#predictions=loaded_model.predict(new_data)
#print(predictions,testY)
#

In [ ]:
import joblib
#joblib.dump(models['GradientBoosting'],'GradientBoostingDailyResourceConsumptionEstimator.pkl')


loaded_model=joblib.load('GradientBoostingDailyResourceConsumptionEstimator.pkl')

new_data=testX
predictions=loaded_model.predict(new_data)
print(predictions,testY)